In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import calendar

## Exploratory Data Analysis & Cleaning

In [2]:
ufo = pd.read_csv('ufo_sightings_dirty.csv', dtype={'duration (seconds)': 'str'})
ufo.head(5)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611


In [3]:
# Break apart datetime to be time, day, month, year

ufo[['date', 'time']] = ufo['datetime'].str.split(' ',expand=True)
ufo[['year', 'month', 'day']] = ufo['date'].str.split('-', expand =True)
ufo[['hour', 'minute','second']] = ufo['time'].str.split(':',expand=True)

# Change capitization of addresses to proper format

ufo['city'] = ufo['city'].str.title()
ufo['shape'] = ufo['shape'].str.title()
ufo['state'] = ufo['state'].str.upper()
ufo['country'] = ufo['country'].str.upper()
ufo['country'] = ufo['country'].replace('AU', 'AUS')
ufo['country'] = ufo['country'].replace('CA', 'CAN')
ufo['country'] = ufo['country'].replace('DE', 'DEU')
ufo['country'] = ufo['country'].replace('GB', 'GBR')
ufo['country'] = ufo['country'].replace('US', 'USA')

# Convert columns to proper formatting

ufo.columns = ufo.columns.str.strip()
ufo['date posted'] = pd.to_datetime(ufo['date posted'])
ufo['duration (seconds)'] = pd.to_numeric(ufo['duration (seconds)'], errors='coerce')
ufo['longitude'] = pd.to_numeric(ufo['longitude'], errors='coerce')
ufo['latitude'] = pd.to_numeric(ufo['latitude'], errors='coerce')
ufo['date'] = pd.to_datetime(ufo['date'])
ufo['year'] = pd.to_numeric(ufo['year'], errors='coerce')
ufo['month'] = pd.to_numeric(ufo['month'], errors='coerce')
ufo['day'] = pd.to_numeric(ufo['day'], errors='coerce')
ufo['hour'] = pd.to_numeric(ufo['hour'], errors='coerce')
ufo['minute'] = pd.to_numeric(ufo['minute'], errors='coerce')

# Create column that shows Month abb and day

ufo['month_abbr'] = ufo['month'].apply(lambda x: calendar.month_abbr[x])
ufo['month_day'] = ufo['month_abbr'].map(str) + ' ' + ufo['day'].map(str)
ufo = ufo.drop(columns = ['second','month_abbr'])
ufo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              80332 non-null  object        
 1   city                  80332 non-null  object        
 2   state                 74535 non-null  object        
 3   country               70662 non-null  object        
 4   shape                 78400 non-null  object        
 5   duration (seconds)    80329 non-null  float64       
 6   duration (hours/min)  80332 non-null  object        
 7   comments              80317 non-null  object        
 8   date posted           80332 non-null  datetime64[ns]
 9   latitude              80332 non-null  float64       
 10  longitude             80332 non-null  float64       
 11  date                  80332 non-null  datetime64[ns]
 12  time                  80332 non-null  object        
 13  year            

In [4]:
# Create new columns that contains date differences for sighting vs reporting

ufo['diff_weeks'] = round((ufo['date posted'] - ufo['date']) / np.timedelta64(1, 'W'))
ufo['diff_months'] = round((ufo['date posted'] - ufo['date']) / np.timedelta64(1, 'M'))
ufo['diff_years'] = round((ufo['date posted'] - ufo['date']) / np.timedelta64(1, 'Y'))

In [5]:
#Calculate total null values

print("--------------\nMissing values: \n{}".format(pd.isnull(ufo).sum()))

--------------
Missing values: 
datetime                   0
city                       0
state                   5797
country                 9670
shape                   1932
duration (seconds)         3
duration (hours/min)       0
comments                  15
date posted                0
latitude                   0
longitude                  0
date                       0
time                       0
year                       0
month                      0
day                        0
hour                       0
minute                     0
month_day                  0
diff_weeks                 0
diff_months                0
diff_years                 0
dtype: int64


In [6]:
#Create season dictionary and then assign numeric month to a specific season

season_dict = {1: 'Winter',2: 'Winter',3: 'Spring', 4: 'Spring',5: 'Spring',6: 'Summer',
               7: 'Summer',8: 'Summer',9: 'Fall',10: 'Fall',11: 'Fall',12: 'Winter'}
ufo['season'] = ufo['month'].apply(lambda x: season_dict[x])
ufo.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,...,year,month,day,hour,minute,month_day,diff_weeks,diff_months,diff_years,season
0,1949-10-10 20:30:00,San Marcos,TX,USA,Cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,...,1949,10,10,20,30,Oct 10,2846.0,655.0,55.0,Fall
1,1949-10-10 21:00:00,Lackland Afb,TX,NaN,Light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,...,1949,10,10,21,0,Oct 10,2932.0,674.0,56.0,Fall
2,1955-10-10 17:00:00,Chester (Uk/England),NaN,GBR,Circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,...,1955,10,10,17,0,Oct 10,2728.0,627.0,52.0,Fall
3,1956-10-10 21:00:00,Edna,TX,USA,Circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,...,1956,10,10,21,0,Oct 10,2466.0,567.0,47.0,Fall
4,1960-10-10 20:00:00,Kaneohe,HI,USA,Light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,...,1960,10,10,20,0,Oct 10,2258.0,519.0,43.0,Fall


In [7]:
# Set up bins and names for timeframe. (Midnight -> 4am returns 'Late Night')

b = [0,4,8,12,16,20,24]
l = ['Late Night', 'Early Morning','Morning','Noon','Evening','Night']

#Create a column for time of day
ufo['time of day'] = pd.cut(ufo['hour'], bins=b, labels=l, include_lowest=True)
ufo.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,...,month,day,hour,minute,month_day,diff_weeks,diff_months,diff_years,season,time of day
0,1949-10-10 20:30:00,San Marcos,TX,USA,Cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,...,10,10,20,30,Oct 10,2846.0,655.0,55.0,Fall,Evening
1,1949-10-10 21:00:00,Lackland Afb,TX,NaN,Light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,...,10,10,21,0,Oct 10,2932.0,674.0,56.0,Fall,Night
2,1955-10-10 17:00:00,Chester (Uk/England),NaN,GBR,Circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,...,10,10,17,0,Oct 10,2728.0,627.0,52.0,Fall,Evening
3,1956-10-10 21:00:00,Edna,TX,USA,Circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,...,10,10,21,0,Oct 10,2466.0,567.0,47.0,Fall,Night
4,1960-10-10 20:00:00,Kaneohe,HI,USA,Light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,...,10,10,20,0,Oct 10,2258.0,519.0,43.0,Fall,Evening


# Fill missing values for 'country'

In [8]:
# Create US state list, filter to see rows with US states and NaN as country and fill with 'US'

ufo_clone = ufo[:]
ufo_state = ufo_clone[ufo_clone['state'].notnull() & ufo_clone['country'].isnull()]
us_states = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA',
            'ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR',
            'PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','PR']
ufo_state = ufo_state[ufo_state['state'].isin(us_states)]
ufo_state['country'].fillna('USA', inplace = True)

# Repeat for filling missing values for Canada

ufo_not_us = ufo_clone[ufo_clone['state'].notnull() & ufo_clone['country'].isnull()]
can_prov = ['AB','ON','BC','MB','NF','NS','QC','YT','NB','PQ','NT','PE','SK','SA']
ufo_not_us = ufo_not_us[ufo_not_us['state'].isin(can_prov)]
ufo_not_us['country'].fillna('CAN', inplace = True)

In [9]:
# Update missing NaN values in un-filtered DF

ufo_clone.update(ufo_state)
ufo_clone.update(ufo_not_us)
ufo_clone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              80332 non-null  object        
 1   city                  80332 non-null  object        
 2   state                 74535 non-null  object        
 3   country               77071 non-null  object        
 4   shape                 78400 non-null  object        
 5   duration (seconds)    80329 non-null  float64       
 6   duration (hours/min)  80332 non-null  object        
 7   comments              80317 non-null  object        
 8   date posted           80332 non-null  datetime64[ns]
 9   latitude              80332 non-null  float64       
 10  longitude             80332 non-null  float64       
 11  date                  80332 non-null  datetime64[ns]
 12  time                  80332 non-null  object        
 13  year            

In [10]:
# Find find countries with missing state data
# Convert long/lat into point format to be compared to geopandas world df

ufo_other = ufo_clone[ufo_clone['state'].isnull() & ufo_clone['country'].isnull()]
ufo_other = gpd.GeoDataFrame(ufo_other, geometry=gpd.points_from_xy(ufo_other.longitude, ufo_other.latitude))
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ufo_other.crs = 'EPSG:4326'

# Join world df and ufo dataframe

ufo_other1 = gpd.sjoin(ufo_other, world, how='left')
ufo_other['country'] = ufo_other1['iso_a3']

# Update original df and fix Kosovo showing -99

ufo_clone.update(ufo_other)
ufo_clone['country'] = ufo_clone['country'].replace('-99', 'XKX')

In [11]:
print("--------------\nMissing values: \n{}".format(pd.isnull(ufo_clone).sum()))

--------------
Missing values: 
datetime                   0
city                       0
state                   5797
country                  395
shape                   1932
duration (seconds)         3
duration (hours/min)       0
comments                  15
date posted                0
latitude                   0
longitude                  0
date                       0
time                       0
year                       0
month                      0
day                        0
hour                       0
minute                     0
month_day                  0
diff_weeks                 0
diff_months                0
diff_years                 0
season                     0
time of day                0
dtype: int64


In [12]:
# Remove rows where all country is missing

ufo = ufo_clone[ufo_clone['country'].notnull()]
ufo.to_csv('ufo_sightings_clean.csv',index=False)